In [1]:
import pandas as pd

Quiero identificar que producen los investigadores de una institución. 

Para eso se propone, identificar primero quienes son esos investigadores que declararon como su filiación, la institución que nos interesa en algun de sus publicaciones. 


In [4]:
sat_openalex_work = catalog.load('stg_openalex/sat_openalex_work')
hub_openalex_work = catalog.load('stg_openalex/hub_openalex_work')

fact_publication_openalex = pd.merge(
    hub_openalex_work,
    sat_openalex_work
    ).drop(columns=['load_datetime','source', 'hashdiff'])

fact_publication_openalex[['title','publication_year','cited_by_count','oa_status']].sort_values(by='cited_by_count', ascending=False)

[03/10/25 13:23:17] INFO     Loading data from stg_openalex/sat_openalex_work                   ]8;id=285950;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=441976;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 13:23:23] INFO     Loading data from stg_openalex/hub_openalex_work                   ]8;id=231215;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=584661;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

,title,publication_year,cited_by_count,oa_status
29650,The ATLAS Experiment at the CERN Large Hadron ...,2008,3806,green
45876,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,2005,3199,bronze
41709,The clustering of galaxies in the completed SD...,2017,2582,bronze
33456,The ATLAS Experiment at the CERN Large Hadron ...,2020,2468,closed
14798,Genome sequence and analysis of the tuber crop...,2011,1979,hybrid
...,...,...,...,...
50063,Effect of tides on the orbital evolution of th...,2021,0,closed
50062,El camino hacia nuevos paradigmas en la resolu...,2010,0,closed
50061,Reconstrucción de la memoria Colectiva: Madres...,2013,0,closed
50059,"Países en vías de Desarrollo: Empleo Público, ...",2017,0,diamond


Merge con OpenAIRE

In [31]:
hub_openalex_doi = catalog.load('stg_openalex/hub_openalex_doi')
sal_openalex_work = catalog.load('stg_openalex/sal_openalex_work')
hub_openaire_researchproduct = catalog.load('stg_openaire/hub_openaire_researchproduct')
link_openaire_researchproduct_doi = catalog.load('stg_openaire/link_openaire_researchproduct_doi')
sat_openaire_researchproduct = catalog.load('stg_openaire/sat_openaire_researchproduct')

[03/10/25 13:55:47] INFO     Loading data from stg_openalex/hub_openalex_doi                    ]8;id=926810;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=113697;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 13:55:48] INFO     Loading data from stg_openalex/sal_openalex_work                   ]8;id=929793;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=646702;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 13:55:49] INFO     Loading data from stg_openaire/hub_openaire_researchproduct        ]8;id=250401;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=781338;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 13:55:51] INFO     Loading data from stg_openaire/link_openaire_researchproduct_doi   ]8;id=358407;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=305862;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 13:55:53] INFO     Loading data from stg_openaire/sat_openaire_researchproduct        ]8;id=33601;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=95358;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

In [43]:
fact_publication = pd.merge(
    fact_publication_openalex[['work_hk','title','publication_year','cited_by_count','oa_status']],
    sal_openalex_work[['work_hk','doi_hk']]
)

fact_publication = pd.merge(
    fact_publication,
    hub_openalex_doi[['doi_hk','doi']]
)

fact_publication = pd.merge(
    fact_publication,
    link_openaire_researchproduct_doi,
    on="doi_hk",
    how='left'
)

fact_publication = pd.merge(
    fact_publication,
    sat_openaire_researchproduct,
    on="researchproduct_hk",
)

fact_publication[['title','publication_year','cited_by_count','oa_status','downloads','views']].sort_values('cited_by_count', ascending=False)


,title,publication_year,cited_by_count,oa_status,downloads,views
27960,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,2005,3199,bronze,87,15
25430,The clustering of galaxies in the completed SD...,2017,2582,bronze,47,12
9042,Genome sequence and analysis of the tuber crop...,2011,1979,hybrid,736,628
12226,The Simons Genome Diversity Project: 300 genom...,2016,1474,green,45,18
1935,The role of constraints within generalized non...,1998,1340,closed,0,0
...,...,...,...,...,...,...
30437,Tecnologías modulares de climatización solar p...,2018,0,diamond,0,0
48,Performance Comparison of Parallel Programming...,2012,0,green,57,19
46,Variación semianual en flujos magnetosféricos ...,2020,0,hybrid,0,0
45,Resultados De La Implementación De Una Estrate...,2017,0,bronze,2,1


# OpenAlex

In [ ]:
pd.merge(
    sal_openalex_work,
    sal_openaire_researchproduct,
    
    )

,work_ids_hk,work_hk,doi_hk,mag_hk,pmid_hk,pmcid_hk,load_datetime,source
0,b'\x00\x02m6\xaa\xc7\x11i\x17 h\x03\xea\xa5\xb...,"b'\xdd\x0c,\xb5\x0f\xf4\x8f\xf0C\xb9\xb9\xfa\x...",b'a\xb2\x83\x07`\xfd\x84X\x07\xfd\xb0T\x16R\x7...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
1,b'\x00\x03>\xca\xae\xb6\x90\xa5\x9aqdG\x12\x9f...,b'\x1c\x9d\xf4\x9e\xf1(g\x8d3p\xda\xb3>\x0b\x1...,"b'\xdcF\x87\r\xd2<\x97\x02\x02""\xb5\xc1JZ\x94b'",b'\x9b\xf0f\x1b\xf7\x8e\xac2\x10B)h\xf9\xe9\xc7Z',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
2,b'\x00\x04\xc7\xfe\xd4\xa8S\xd7\xa2M\xbb\x7fC\...,b'X\x01\xbc\xc7\x80\xe6\xf8\x82\xf2\x946\x1eL/...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xe2g4\xd15>I\xe9\xe7A\x0b\xf0\xe3\x87KZ',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
3,b'\x00\x06*Ua!\xb8\x8e(\x02\x99\x1c\xbe \xe3\xc8',b'~\x86c\x87Do@\x8dW\xdb\xcc\xd8U\xb6kh',b'\x13k\xa2\x80\x1b\xaf\xa0\xbd/\xcb<`\xae\xf7...,b'_\xdbb\xd0\xceo\x9eM`\x01DG\xa0\xadA\xce',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
4,b'\x00\x06?Y\xe8S++\x1f~\xf2\x82\\k4\xf7',b'[tS\xb0W\x1a\xe0\xa9e\xd8\xec.\xd4W\xb0\x9c',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\x8a\xd1j5I}\xfbZ\xa7Np+\xf6w\xfa\xba',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
...,...,...,...,...,...,...,...,...
50087,b'\xff\xf9\xa7\xd6@GA #\x9c\x1b9\x8e-\xc1)',b'Hc\xa9\x1c\xbf\xc5\n\xd2^\x9f\xab\xd4\xb6\x1...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\x17\xd0&\x16r$\xa0\xd4*HK\xb3\xaa\xddq\xab',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
50088,b'\xff\xf9\xb1\x94$\x86\x88\xf6\xd0\x88\xcd\x1...,b'z9Ma\xe6-\x10K^\xfa%\xa7\x9d-c\xa4',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\x9a\xec$\x8b\xcc\xde\xc6!x\xd8\x9a\xf1\x85\...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
50089,b'\xff\xfd1\xd0h\xd8HJ\xcf\x05(%\xee\x90.y',b'\xdd\xfdc\x1fW[\x02\xc4~N\x05{\xeeT\xe0\x1b',b'\xd4\xbe\x9d\xc6\xa9\xa6\x18\xcf$\xffi<\x89\...,b'\xcd\n\\5\xd0\xc6\xbf\xc4\x11\xbd\xd6\xac\xe...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX
50090,b'\xff\xfdM\xf6+I\x0e\xac8)}\x9d\xc9R\xae\xab',b'\xe9\t\xdf\xa7fC\x93H\x03\x1a{~D\x05\x9a\xe7',b'\x11\xc1\xea\xeaU\xbb$M\x84uk\x8b\xad?J\xfc',b'\x8dv\x1f\xc3\xb1:(\xa4\xad\x9b9n\x9eq^\xeb',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-02 12:53:58.781712,OPENALEX


### ORCID

ORCID es un PID, y eso tiene sus beneficios sobre todo en términos de calidad de datos. También hay que tener en cuenta que no todos los investigadores tiene ORCID. Y de donde se recuperan esos ORCID. 

Este enfoque busca mostrar la potencialidad del uso de PID como ORCID y hacer una aproximación a algo realmente beneficioso y mostrar utilidad para quienes tienen que tomar decisiones en el ámbito científico académico.

Tengo ORCID de autores en OpenAlex y datos sobre investigadores en OpenAIRE

Los autores de OpenAlex se recuperaron utilizando un filtro

Los de OpenAIRE tiene un endpoint para recuperar investigadores directamente, como OpenAlex que le dedica un endpoint exclusivo para ese tipo de entidades. Si se puede recuperar autores a través de recursos recuperados del endpoint resarchproduct/. Existe un filtro relOrganizationId	que recupera _research products_ conectados a una organización, lo que nos permite recuperar todos los autores de las publicaciones, donde al menos exista un autore institucional, lo que no quiere decir que necesariamente todos los autores sean de la institución en cuestión.

Para procesar estos datos, se crearon unas tablas dedicadas a almacenar los ORCID de cada fuente. Asi despues puedo armar una dim_author, a partir de otras tablas contextuales, llamadas satélites.

In [ ]:
# UNLP ror
institution_ror = catalog.load('params:openalex_fetch_options.institution_ror')

# Cargar hubs de autores (solo contienen los identificadores)
hub_openalex_orcid = catalog.load('stg_openalex/hub_openalex_orcid')
hub_openaire_orcid = catalog.load('stg_openaire/hub_openaire_orcid')

# Cargar satélites con metadatos de autores
sat_openalex_author = catalog.load('stg_openalex/sat_openalex_author')
sat_openaire_researchproduct = catalog.load('stg_openaire/sat_openaire_researchproduct')

sal_openalex_author = catalog.load('stg_openalex/sal_openalex_author')

sat_openaire_author = catalog.load('stg_openaire/sat_openaire_author')
#sal_openaire_author = catalog.load('stg_openaire/sal_openaire_author')


[03/10/25 09:12:02] INFO     Loading data from params:openalex_fetch_options.institution_ror    ]8;id=303305;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=906648;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from stg_openalex/hub_openalex_orcid                  ]8;id=239357;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=142388;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

                    INFO     Loading data from stg_openaire/hub_openaire_orcid                  ]8;id=26208;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=91128;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 09:12:03] INFO     Loading data from stg_openalex/sat_openalex_author                 ]8;id=519591;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=901434;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

                    INFO     Loading data from stg_openaire/sat_openaire_researchproduct        ]8;id=991037;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=766048;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/10/25 09:12:09] INFO     Loading data from stg_openalex/sal_openalex_author                 ]8;id=367317;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=271013;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

                    INFO     Loading data from stg_openaire/sat_openaire_author                 ]8;id=655117;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=385926;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

In [28]:
links_openaire_researchproduct_author = catalog.load('stg_openaire/links_openaire_researchproduct_author ')


[03/10/25 09:29:02] INFO     Loading data from                                                  ]8;id=74697;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=224156;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             stg_openaire/links_openaire_researchproduct_author                                    
                             (SQLTableDataset)...                                                                  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/pablo/.local/lib/python3.10/site-packages/kedro/io/core.py:244 in load                     │
│                                                                                                  │
│   241 │   │   │   self._logger.debug("Loading %s", str(self))                                    │
│   242 │   │   │                                                                                  │
│   243 │   │   │   try:                                                                           │
│ ❱ 244 │   │   │   │   return load_func(self)                                                     │
│   245 │   │   │   except DatasetError:                                                           │
│   246 │   │   │   │   raise                                                                      │
│   247 │   │   │   except Exception as exc:                                                       │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/kedro_datasets/pandas/sql_dataset.py:266 in load │
│                                                                                                  │
│   263 │   │   }                                                                                  │
│   264 │                                                                                          │
│   265 │   def load(self) -> pd.DataFrame:                                                        │
│ ❱ 266 │   │   return pd.read_sql_table(con=self.engine, **self._load_args)                       │
│   267 │                                                                                          │
│   268 │   def save(self, data: pd.DataFrame) -> None:                                            │
│   269 │   │   data.to_sql(con=self.engine, **self._save_args)                                    │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/io/sql.py:386 in read_sql_table           │
│                                                                                                  │
│    383 │                                                                                         │
│    384 │   with pandasSQL_builder(con, schema=schema, need_transaction=True) as pandas_sql:      │
│    385 │   │   if not pandas_sql.has_table(table_name):                                          │
│ ❱  386 │   │   │   raise ValueError(f"Table {table_name} not found")                             │
│    387 │   │                                                                                     │
│    388 │   │   table = pandas_sql.read_table(                                                    │
│    389 │   │   │   table_name,                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Table links_openaire_researchproduct_author  not found

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 links_openaire_researchproduct_author = catalog.load('stg_openaire/links_openaire_resear     │
│   2                                                                                              │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py:396 in load             │
│                       

El hub de ORCID de Openalex tiene PIDs de autores de la institución, ya que fueron recuperados a través del endpoint [/authors](https://api.openalex.org/authors) y el filtro [affiliations.institution.ror](https://docs.openalex.org/api-entities/authors/filter-authors)

..

In [23]:
hub_openalex_orcid


,orcid_hk,orcid,load_datetime,source
0,b'\x00\x1f\x8f\xc1\x9b\xd3;o\xb4h\t\x97f\xd1\x...,0000-0002-4173-8648,2025-03-06 08:46:53.860105,OPENALEX
1,"b'\x00""\x8c\x9cP\xa2\x13lo\xf5\x18\x9d\xdf\x08...",0000-0002-7987-4418,2025-03-06 08:46:53.860105,OPENALEX
2,b'\x00#\x16Y\x02\xee\x83\x95\xbcLr\x81?\xb4\x86J',0000-0003-2033-7949,2025-03-06 08:46:53.860105,OPENALEX
3,b'\x006\xa6\xf3E\x805\xf34\xcf\xdf\xee5a\xd6\xb4',0000-0002-9579-7096,2025-03-06 08:46:53.860105,OPENALEX
4,b'\x009\xb3\xd5\x14\x85\xdfN\x14\xd9\x1a+n\x96...,0000-0002-5423-1846,2025-03-06 08:46:53.860105,OPENALEX
...,...,...,...,...
6653,"b'\xff\xe1\x89[\x15\xcf0""\xf7\x18\xeeu\xc2f\xd1`'",0000-0001-8652-9172,2025-03-06 08:46:53.860105,OPENALEX
6654,b'\xff\xe2\x11\xb7\xb5\x02J^Oj\x19}\x94g\xc2\x03',0000-0002-2732-5302,2025-03-06 08:46:53.860105,OPENALEX
6655,b'\xff\xe9xtYU\xdbT\xb0T7t\x04\xcc\xcd\xf6',0000-0002-8931-6831,2025-03-06 08:46:53.860105,OPENALEX
6656,b'\xff\xeb\xe2\x08?)3s\x1cP*\xb2\xac\xd0;>',0000-0002-1342-1598,2025-03-06 08:46:53.860105,OPENALEX


El hub de OpenAIRE tiene ORCID se pobló con datos de todos los autores de [researchproducts](https://graph.openaire.eu/docs/data-model/entities/research-product/), que pueden haber tenido o no la filiación institucional que nos interesa  

In [9]:
hub_openaire_orcid

,orcid_hk,orcid,load_datetime,source
0,"b'\x00\x00\xbe\x16,\xec\xe7\xfaC\xad@\xf7@\xd4...",0000-0003-0631-4775,2025-02-27 15:00:00,OPENAIRE
1,b'\x00\x05h\x83-\xa2\xbd\xdc\xf7\xb7\rR\xc5\xc...,0000-0001-8119-0333,2025-02-27 15:00:00,OPENAIRE
2,b'\x00\x08K@\x9e\x12\xfb\x934\xa65\xdd\x88h~V',0000-0001-8419-2473,2025-02-27 15:00:00,OPENAIRE
3,b'\x00\n\x1f\xcc\xef}[=\xd2j\xd5\xbb\xa2qY\xb1',0000-0002-8399-9993,2025-02-27 15:00:00,OPENAIRE
4,b'\x00\x13\x1e\xf2\xbc\x903\xf7\xcd\xc1G\xfeD\...,0000-0002-6236-2934,2025-02-27 15:00:00,OPENAIRE
...,...,...,...,...
26263,b'\xff\xf1\x04\xec\xd9C\xfd5Q\x8d\x10Y\xd5\xb2`-',0000-0001-9211-7019,2025-02-27 15:00:00,OPENAIRE
26264,b'\xff\xf1}\xa5\x02i\xa4\r\x07\xb2\x8f\xb0XF\x...,0000-0001-8487-3594,2025-02-27 15:00:00,OPENAIRE
26265,b'\xff\xf6O\xcd\xf0\xe7\x1c^va M\xc6k\xdf\xdd',0000-0001-6331-3272,2025-02-27 15:00:00,OPENAIRE
26266,b'\xff\xf7\x9fC\xc4\xc69o\xcd\xd0c\x1d\xbb<\xb...,0000-0002-7516-5286,2025-02-27 15:00:00,OPENAIRE


Puedo unir las tablas a partir del orcid_hk y quedarme con los autores institucionales, y evitar los autores que no necesariamente son institucionales asi:

In [27]:
inst_authors = pd.merge(
    hub_openalex_orcid, 
    hub_openaire_orcid, 
    on='orcid_hk', 
    how='left',
    suffixes=('','_openaire')
    )


# Filtrar las columnas que terminan con '_openaire'
columnas_a_eliminar = inst_authors.filter(regex='_openaire$').columns

# Eliminar las columnas
inst_authors = inst_authors.drop(columnas_a_eliminar, axis=1)

inst_authors

,orcid_hk,orcid,load_datetime,source
0,b'\x00\x1f\x8f\xc1\x9b\xd3;o\xb4h\t\x97f\xd1\x...,0000-0002-4173-8648,2025-03-06 08:46:53.860105,OPENALEX
1,"b'\x00""\x8c\x9cP\xa2\x13lo\xf5\x18\x9d\xdf\x08...",0000-0002-7987-4418,2025-03-06 08:46:53.860105,OPENALEX
2,b'\x00#\x16Y\x02\xee\x83\x95\xbcLr\x81?\xb4\x86J',0000-0003-2033-7949,2025-03-06 08:46:53.860105,OPENALEX
3,b'\x006\xa6\xf3E\x805\xf34\xcf\xdf\xee5a\xd6\xb4',0000-0002-9579-7096,2025-03-06 08:46:53.860105,OPENALEX
4,b'\x009\xb3\xd5\x14\x85\xdfN\x14\xd9\x1a+n\x96...,0000-0002-5423-1846,2025-03-06 08:46:53.860105,OPENALEX
...,...,...,...,...
6653,"b'\xff\xe1\x89[\x15\xcf0""\xf7\x18\xeeu\xc2f\xd1`'",0000-0001-8652-9172,2025-03-06 08:46:53.860105,OPENALEX
6654,b'\xff\xe2\x11\xb7\xb5\x02J^Oj\x19}\x94g\xc2\x03',0000-0002-2732-5302,2025-03-06 08:46:53.860105,OPENALEX
6655,b'\xff\xe9xtYU\xdbT\xb0T7t\x04\xcc\xcd\xf6',0000-0002-8931-6831,2025-03-06 08:46:53.860105,OPENALEX
6656,b'\xff\xeb\xe2\x08?)3s\x1cP*\xb2\xac\xd0;>',0000-0002-1342-1598,2025-03-06 08:46:53.860105,OPENALEX


In [11]:
# Unir autores de OpenAlex y OpenAIRE
df_authors = pd.merge(
    hub_openalex_orcid[['orcid_hk', 'orcid']],
    hub_openaire_orcid[['orcid_hk', 'orcid']],
    on='orcid_hk',
    how='outer',  # Outer para no perder autores que solo están en una fuente
    suffixes=('_openalex', '_openaire')
)

df_authors


,orcid_hk,orcid_openalex,orcid_openaire
0,"b'\x00\x00\xbe\x16,\xec\xe7\xfaC\xad@\xf7@\xd4...",NaN,0000-0003-0631-4775
1,b'\x00\x05h\x83-\xa2\xbd\xdc\xf7\xb7\rR\xc5\xc...,NaN,0000-0001-8119-0333
2,b'\x00\x08K@\x9e\x12\xfb\x934\xa65\xdd\x88h~V',NaN,0000-0001-8419-2473
3,b'\x00\n\x1f\xcc\xef}[=\xd2j\xd5\xbb\xa2qY\xb1',NaN,0000-0002-8399-9993
4,b'\x00\x13\x1e\xf2\xbc\x903\xf7\xcd\xc1G\xfeD\...,NaN,0000-0002-6236-2934
...,...,...,...
28517,b'\xff\xf1\x04\xec\xd9C\xfd5Q\x8d\x10Y\xd5\xb2`-',NaN,0000-0001-9211-7019
28518,b'\xff\xf1}\xa5\x02i\xa4\r\x07\xb2\x8f\xb0XF\x...,NaN,0000-0001-8487-3594
28519,b'\xff\xf6O\xcd\xf0\xe7\x1c^va M\xc6k\xdf\xdd',NaN,0000-0001-6331-3272
28520,b'\xff\xf7\x9fC\xc4\xc69o\xcd\xd0c\x1d\xbb<\xb...,0000-0002-7516-5286,0000-0002-7516-5286


In [15]:
sal_openalex_author

,author_orcid_hk,author_hk,orcid_hk,load_datetime,source
0,b'\x00\x04\x06\x00\x8d\xd1\xe7\xbc\xb6\xd0K\xd...,b'D\xfb\xfc\x06\x98\x8a\x02P\x1b\x8e\x8d\xff\x...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX
1,b'\x00\x05\x1aH\xb6\x02N\x0fR\xf7e\x0e\xd2\xc5...,"b']\x7f\xc3x\xcf\xb7\x1aHy\xb6w*""\x97\xc69'",b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX
2,"b'\x00\x07E]\x17\xb8Y:\xc8\xc4""\xac\x0b\x19\x1...",b'\xb7ZK\xa0X\x06\xb0\xf2\x02\x1e\xd8\xe7\x02\...,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX
3,"b'\x00\x10\x82\x12-\xd7\xefs\xb8p\xd1\xa2\x06""...","b""=\x9c\x17\xd4\x10c%'\x89\x1ax\x08\xcf\x8e\xe...",b'G\xfa\xa9\xec\xe9\xa0BS%\x03\xe2\\\xa4\xc2$\...,2025-03-06 08:46:53.860105,OPENALEX
4,"b'\x00\x16[:\xed\x19L\x1b\xb084]+\xb5%,'","b""\xf5\xb5\xb9\x16\xa5q9\xc9\x83\x92T\xd6\x1e'...",b'S\x87`3\x1f\xaa\xf8\xcfn\xd7\x99%z\xe4\x8c?',2025-03-06 08:46:53.860105,OPENALEX
...,...,...,...,...,...
20215,b'\xff\xf4\xa9\xfb\x98\xc4\x93#\xec\xb5\xf9\\\...,b'\xaf\x0e\x88\x8e}N/\xb3!\x0c\xe8t\xe4\x8b\xc1b',b'cqh\x90\x7f\xd6\x86\xa3v\xbf\xf1\xf0[\x1f!h',2025-03-06 08:46:53.860105,OPENALEX
20216,b'\xff\xf6\xb3\x89\xb4V\x9e\xbe\xee\xcf\xf2\x0...,"b'""\xf5\xc7\xd5\xd8#\xde\xaam\xedE2|\xb3hZ'",b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX
20217,b'\xff\xf74\nQj\xc4\xf3`\xda\x9a\x01{\xcaj ',b'\x9e\x9c\x8a\xeb?\x1dZ\xc5\xf7\xc30\x06>Y5\xd1',b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX
20218,b'\xff\xf8\xd9\xdal$\x9e<9(\xa0<\xf8\xf1\xd7\x04',"b'Y\x0c\x8a\x9f\xdb\x1f\xe9,E\xc5\x95\x82aC\xb6x'",b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#',2025-03-06 08:46:53.860105,OPENALEX


In [16]:
pd.merge(
    sat_openalex_author,
    sal_openalex_author[['author_hk', 'orcid_hk']],
    on="author_hk")

,author_hk,hashdiff,author_id,orcid,display_name,works_count,cited_by_count,load_datetime,source,orcid_hk
0,b'\xa8\xdf\x99\xe0\xc4\x0b#\x9542J\xca=\x93eQ',b'\x99mO\xbd\xb2\x15}\x13\x83\xe7\xfdq\x85vkl',https://openalex.org/A5027025610,https://orcid.org/0000-0002-3614-807X,Nicolás Toro,138,3876,2025-03-06 08:46:53.860105,OPENALEX,b'\xe7\xa1`\xb4\rj\xee1!K\xad\x96\x8c\xa4/\xd7'
1,b'\xbc\xc4\xdd\x02\x1a\x19\xf6\x0e\x03\xef5\xa...,b'\xb8b\x7f2ct\xe8\xc3[\x82e\x15\x1f[\x9a\xbe',https://openalex.org/A5030527440,NO DATA,Verónica It,8,58,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'
2,"b'Q\t\x96Y\xaa]-,\xb8e\x95\xdc\xe8\xb0ba'",b'\x03\xaeP\xd7\xc7-LP9D\xa2\x1a\xce\xe0o\xbf',https://openalex.org/A5085232864,NO DATA,Juan B. Rossi,2,8,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'
3,b'\x0e\xd0\x93\xcdf\xb5&\x0c\xf4\xd3\xc1\xf5\x...,b'\xa7_\xa8D\x03\xe1\x88\x9f\xcet\x95\x02\xc7r...,https://openalex.org/A5005299246,https://orcid.org/0000-0002-8179-3406,Alejo Carlos Scarano,28,217,2025-03-06 08:46:53.860105,OPENALEX,b'\x06\xec\xf3\x03pY\x80Px\xab\x050\xe3\xb8\xa33'
4,b')\xdf\x07K\xc2\x1b\xa6\xe8*.$\xa6\xda\xc1{!',b'\xe3Q?\x9cF\xf0\xfc\xa7\xef\xccj5\xaba\x1bM',https://openalex.org/A5070601695,NO DATA,Horacio Villagarcía Wanza,1,1,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'
...,...,...,...,...,...,...,...,...,...,...
20215,"b'\xeci\x83\x95C\xaf\xc7""G.\x1a\xe0\xd6\xbaX\x18'","b'2\x04\xe5U""p\x06\xe5\xab\xcd[?\x87~\xe1\xba'",https://openalex.org/A5092504139,NO DATA,CAROLINA ACOSLA HOSPILALECHE,2,3,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'
20216,b'\xd4\xe0\xef\xf2\xadqs\xb3;\xbaSb\x04\x1a\xa...,b'T\xde\xb0>\xc4@6\\c)\x16\x8d\xbd7B?',https://openalex.org/A5070446799,https://orcid.org/0000-0002-2260-2825,Juan Manuel Bellini,5,0,2025-03-06 08:46:53.860105,OPENALEX,b'\x16\x8f\xdd\xd9~>?\x8a\xbe\xec\xba?\xd2I{\xde'
20217,b'\xa3\x861\xd6\x90T\xb3ZPX\xfb\x80\xdd7\x02&',b'x\xc0M[\x91L\xf9\xd0\xf9\x9d\xfej\x19F\x91\xe6',https://openalex.org/A5021967915,NO DATA,Nestor Hugo Rosendo,1,0,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'
20218,b'\xcez\xd0\x94\xa4\xb3i\xf0\x05\x13\xca\x96\x...,b'\xc6\xba?\x8c\xb0t9T\xf3[\x8aiL\x80\xce\xc9',https://openalex.org/A5055362404,NO DATA,L. Nožka,7,123,2025-03-06 08:46:53.860105,OPENALEX,b'\xc2\xcdi\xfe\x9c\xee\xadn\x8e=&\x82\x12G\xf1#'


In [ ]:
sat_openalex_author = pd.merge(
    sat_openalex_author,
    sal_openalex_author[['author_hk', 'orcid_hk']],
    on="author_hk")

sat_openaire_author = pd.merge(
    sat_openaire_author,
    sal_openaire_author[['author_hk', 'orcid_hk']],
    on="author_hk")

# Agregar metadatos de OpenAlex
df_authors = pd.merge(df_authors, sat_openalex_author, on='orcid_hk', how='left')

# Agregar metadatos de OpenAIRE (si hay datos de autor)
df_authors = pd.merge(df_authors, sat_openaire_researchproduct, on='orcid_hk', how='left')

# Añadir la fuente de procedencia
df_authors['source'] = df_authors.apply(
    lambda row: 'BOTH' if pd.notna(row['orcid_openalex']) and pd.notna(row['orcid_openaire']) 
    else 'OPENALEX' if pd.notna(row['orcid_openalex']) 
    else 'OPENAIRE',
    axis=1
)

# Limpiar nombres de columnas innecesarias
df_authors.drop(columns=['orcid_openalex', 'orcid_openaire'], inplace=True)

# Verificar
print(df_authors.head())


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 df_authors = pd.merge(df_authors, sat_openalex_author, on='orcid_hk', how='left')           │
│    8                                                                                             │
│    9 # Agregar metadatos de OpenAIRE (si hay datos de autor)                                     │
│ ❱ 10 df_authors = pd.merge(df_authors, sat_openaire_researchproduct, on='orcid_hk', how='left    │
│   11                                                                                             │
│   12 # Añadir la fuente de procedencia                                                           │
│   13 df_authors['source'] = df_authors.apply(                                                    │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/reshape/merge.py:170 in merge        │
│                                                                                                  │
│    167 │   │   │   copy=copy,                                                                    │
│    168 │   │   )                                                                                 │
│    169 │   else:                                                                                 │
│ ❱  170 │   │   op = _MergeOperation(                                                             │
│    171 │   │   │   left_df,                                                                      │
│    172 │   │   │   right_df,                                                                     │
│    173 │   │   │   how=how,                                                                      │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/reshape/merge.py:794 in __init__     │
│                                                                                                  │
│    791 │   │   │   self.join_names,                                                              │
│    792 │   │   │   left_drop,                                                                    │
│    793 │   │   │   right_drop,                                                                   │
│ ❱  794 │   │   ) = self._get_merge_keys()                                                        │
│    795 │   │                                                                                     │
│    796 │   │   if left_drop:                                                                     │
│    797 │   │   │   self.left = self.left._drop_labels_or_levels(left_drop)                       │
│                                                                                                  │
│ /home/pablo/.local/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1297 in             │
│ _get_merge_keys                                                                                  │
│                                                                                                  │
│   1294 │   │   │   │   │   │   #  the latter of which will raise                                 │
│   1295 │   │   │   │   │   │   rk = cast(Hashable, rk)                                           │
│   1296 │   │   │   │   │   │   if rk is not None:                                                │
│ ❱ 1297 │   │   │   │   │   │   │   right_keys.append(right._get_label_or_level_values(rk))       │
│   1298 │   │   │   │   │   │   else:                                                             │
│   1299 │   │   │   │   │   │   │   # work-around for merge_